In [1]:
import pandas as pd
import re

In [6]:
li = []

In [7]:
for i in range(1, 71):
    li.append(pd.read_excel('./data/%d.xlsx'%i))

In [9]:
result = pd.concat(li, join='outer',axis=0,ignore_index=True)

In [12]:
result['用途']=result['用途'].astype(str).str.replace('用地','')

In [13]:
result['出让年限']=result['出让年限'].astype(str).str.replace('年','')

In [15]:
result['起始价格（万元）']=result['起始价格（万元）'].astype(str).str.replace('万元','')

In [16]:
result['成交价格（万元）']=result['成交价格（万元）'].astype(str).str.replace('万元','')

In [17]:
result['成交年份']=result['交易时间'].astype(str).str[:4]

In [19]:
result['成交年月']=result['交易时间'].astype(str).str[:8]

In [78]:
def parse_num(num):
    try:
        num = re.match(".*(\d.\d$)", num)[1]
    except:
        pass
    return num
result['容积率（新）'] = result['容积率'].apply(parse_num)

In [116]:
def parse_num(s):
    try:
        s = s.replace('\n','').replace(' ','').replace('\t','')
        s = re.match(r".*限高(\d+)(米|m)", s)[1]
    except:
        try:
            s = re.match(r".*限高(.*)(米|M)", s)[1]
        except:
            try:
                s = re.match(r".*高度(.*?)(米|M)", s)[1]
            except:
                try:
                    s = re.match(r".*高度(:|：)(.*?)(M|m)", s)[2]
                except:
                    try:
                        s = re.match(r".*建筑控制高度：(.*)。", s)[1]
                    except:
                        s = s
    return s
result['限高'] = result['其他'].apply(parse_num)

In [69]:
s = '''1.该宗地位于长沙县空港城小康路以东、高塘路以南，依据湖南省人民政府（2016）政国土字第641号、（2017）政国土字第380号《农用地转用、土地征收审批单》批准转用征收，系国有新征地，现申请挂牌出让。2.根据长沙县自然资源局建设用地规划条件通知单（编号：CXTJ20190919053号）和长沙县自然资源测绘中心提供的宗地图，本次挂牌出让面积42479.39㎡（合63.719亩），用地性质为商务金融用地（B2商务用地），容积率:FAR≤2.0,建筑密度:D%≤25%,绿地率：GRA%≥30%，建筑控制高度：H≤30m且须满足黄花机场限高的相关要求。其他要求详见同时网挂公示的规划条件通知单、调查蓝线图。3.挂牌起始价含：①用地报批应缴规费；②土地出让金；③拆迁补偿安置费；④土地收益等。不包括（不限于）：出让和转让所有税费；挂牌交易服务费；城市基础设施配套费；有线、电讯、光缆、燃气燃油管道、变压器及10000V以上电力线路迁移费等，本次挂牌出让应缴纳的价外税费由竞得人按规定支付。4.付款方式：竞得人应当在签订《成交确认书》之日起7个工作日内与长沙县自然资源局签订

'''

In [139]:
data = pd.read_excel('result.xlsx')

In [135]:
def area(num):
    num = re.match('(.*?)平方米',num)[1]
    return num
data['平方米'] = data['面积（平方米）'].apply(area)

In [136]:
data['建筑面积'] = data['平方米'].mul(data['容积率（新）'])

TypeError: can't multiply sequence by non-int of type 'float'

In [159]:
def high(s):
    try:
        s = str(s)
        s = s.replace('：','').replace('为','').replace('H','').replace('h','').replace('≤','').replace('＜','')
    except:
        pass
    return s
data['限高（新）'] = data['限高'].apply(high)

In [163]:
def deal_price(num):
    try:
        num = str(num).replace(' ','')
        num = re.match('(\d+)\[',num)[1]
    except:
        pass
    return num
data['成交价格'] = data['成交价格（万元）'].apply(deal_price)

In [172]:
data.to_excel('result.xlsx')

In [167]:
data['交易时间'] = data['交易时间'].astype(str).str[:11]

In [171]:
def parse_time(ti):
    ti = ti.replace('年','/').replace('月','/').replace('日','')
    return ti
data['交易时间'] = data['交易时间'].apply(parse_time)

In [3]:
data= pd.read_excel('result.xlsx')

In [5]:
data.rename(columns = {"行政区": "省市", "用途":"土地用途","面积（平方米）":"地块面积（平方米）"},  inplace=True)

In [8]:
data.rename(columns = {"编号2": "地块公告号", "建筑面积":"建筑面积（平方米）"},  inplace=True)

In [9]:
data.columns.to_list()

['编号',
 '省市',
 '地块公告号',
 '区县',
 '宗地名称',
 '宗地位置',
 '土地用途',
 '交易方式',
 '出让年限',
 '起始价格（万元）',
 '受让单位',
 '容积率',
 '交易时间',
 '其他',
 '成交状态',
 '成交年份',
 '成交年月',
 '地块面积（平方米）',
 '建筑面积（平方米）',
 '限高',
 '成交价格（万元）']

In [11]:
result = data[["编号","土地用途", "省市", "区县", "地块公告号", "宗地名称", "宗地位置", "出让年限", "交易方式", "起始价格（万元）", "成交价格（万元）", "成交年份", "成交年月", "交易时间", "地块面积（平方米）", "容积率", "建筑面积（平方米）", "限高", "受让单位","成交状态",'其他']]

In [14]:
result[result.duplicated('宗地位置')]

,编号,土地用途,省市,区县,地块公告号,宗地名称,宗地位置,出让年限,交易方式,起始价格（万元）,...,成交年份,成交年月,交易时间,地块面积（平方米）,容积率,建筑面积（平方米）,限高,受让单位,成交状态,其他
23,72,工业,长沙市,宁乡市,[2019]宁乡市026号,宁乡高新区金水东路以南、欧洲北路以西,宁乡高新区金水东路以南、欧洲北路以西,50,挂牌,10492,...,2019.0,2019年06月,2019/06/18,258399.75,≥1.0,258399.7500,NaN,长沙众迪锂电池有限公司,成交,\n\n\n1.该宗地经省人民政府（2015）政国土字第2083号、（2016）政国土字第6...
45,466,商业,长沙市,宁乡市,[2019]宁乡市020号,夏铎铺镇香山新村岳宁大道旁,夏铎铺镇香山新村岳宁大道旁,40,挂牌,8185,...,2019.0,2019年05月,2019/05/20,34514.82,≤2.0,69029.6400,NaN,长沙炎黄文化科技有限公司,成交,\n\n\n1.该宗地经省人民政府（2018）政国土字第322号审批单批准为国有建设用地，系...
66,584,其他,长沙市,长沙县,[2019]长沙县041号,长沙县开慧镇葛家山村,长沙县开慧镇葛家山村,50,挂牌,1365,...,2019.0,2019年09月,2019/09/02,14214.00,FAR≤2.0,28428.0000,36,长沙县地产投资开发有限公司,成交,\n\n\n1.该宗地位于长沙县开慧镇葛家山村，依据湖南省人民政府（2013）政国土字第12...
113,53,工业,长沙市,开福区,[2019]长沙市080号,开福区金霞开发区,开福区金霞开发区,50（物流仓储）,挂牌,2351,...,2019.0,2019年09月,2019/09/24,18213.33,≤1.0,18213.3300,NaN,长沙深国际综合物流港发展有限公司,成交,\n\n\n1、该宗地位于开福区金霞开发区，总用地面积23799.75㎡，有效面积18213...
203,905,工业,长沙市,高新区,[2019]长沙市085号,高新区枫林路与月季路交汇处西南角,高新区枫林路与月季路交汇处西南角,50,挂牌,7697,...,2019.0,2019年10月,2019/10/09,129129.14,1.0≤低多层≤2.0；1.0≤高层≤3.0,387387.4200,低多层24,长沙汇智新城机械制造有限责任公司,成交,\n\n\n1. 根据长沙高新区土地规划管理协调委员会2019年第7次会议纪要，拟办理高新区...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,1024,商业,长沙市,天心区,[2018]长沙市096号,天心区书院路,天心区书院路,40,挂牌,92000,...,NaN,NaN,NaN,21883.92,≤6.0,131303.5200,NaN,NaN,终止,1、该宗地位于天心区书院路，原为长沙市南湖新城建设开发有限责任公司划拨用地，原核发《国有土地...
1043,1029,商业,长沙市,开福区,[2018]长沙市057号,开福区黄兴北路,开福区黄兴北路,40,挂牌,50295,...,NaN,NaN,NaN,10264.28,≦7.0,71849.9600,NaN,NaN,终止,1、该宗地位于开福区黄兴北路，属长沙市新河三角洲开发建设有限公司棚改前期整理用地（S6地块）...
1044,1035,商业、住宅,长沙市,雨花区,[2018]长土网013号,雨花区新花侯路,雨花区新花侯路,商业40，住宅70,挂牌,300000,...,NaN,NaN,NaN,200272.34,N03-E12≤4.23，N03-E16≤6.0，N03-F022≤4.02,805094.8068,NaN,NaN,中止,1、本宗地有效面积共200272.34㎡，由已收回的国有建设用地和部分国有道路用地两部分构成...
1046,1041,商业、住宅,长沙市,雨花区,[2019]长沙市078号 （恢复挂牌）第二次,雨花区杜花路,雨花区杜花路,商业40，住宅70,挂牌,222390,...,NaN,NaN,NaN,163887.86,详见须知,NaN,不低于249,NaN,未成交,一、该宗地位于雨花区杜花路，高铁站西侧。该宗地对应长国土资政发【2014】100号、长国土资...
